In [1]:
import datasets

in_jsonl = '2_brevity_benchmark/phi3_arena_winners.jsonl'
dataset = datasets.load_dataset('json', data_files=in_jsonl, split='train')
n_rows = len(dataset)
print(str(n_rows))

33000


In [10]:
# Function to filter out duplicates.
def filter_out_duplicates(example, seen_questions = set()):
    if example['question'] in seen_questions:
        return False
    else:
        seen_questions.add(example['question'])
        return True

dataset = dataset.filter(filter_out_duplicates)
len(dataset)

Filter:   0%|          | 0/33000 [00:00<?, ? examples/s]

26968

In [14]:
# Function to filter out errors, normal answers, and 
# short answers that are not much shorter than normal answers.
def filter_out_bad_data(example):
    name_best, ans_best, ans_normal = example['name-best'], example['answer-best'], example['answer-normal']
    # Include only if:
    return name_best!='ERROR' and name_best=='short' and len(ans_best)<0.6*len(ans_normal)

dataset = dataset.filter(filter_out_bad_data)
len(dataset)

21394

In [16]:
out_jsonl = '3_dataset_filter/phi3_arena_final.jsonl'
dataset.to_json(out_jsonl, lines=True)

Creating json from Arrow format:   0%|          | 0/22 [00:00<?, ?ba/s]

21105910